# I rounded the most of the values for better result except polarity,subjectivity and fog index

In [1]:
import requests 
import string
from bs4 import BeautifulSoup

In [2]:
#enter the url here to extract the content and perform data analysis
url = 'https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('title').get_text()

article_text = soup.find(attrs={'class':'td-post-content'})
article=article_text.get_text()


print(title)
print(article)


Challenges and Opportunities of Big Data in Healthcare - Blackcoffer Insights

Big Data
To begin with I shall first like to explain what big data is and why it has become so important in our lives. Big data is simply data, but with a huge size. Data that is not just voluminous but also growing exponentially with time. Such data sets are so large and complex that it is not possible to store or process them using traditional data management tools. So, how huge can this data be? Social media sites like Facebook generate more than 500 terabytes of new data every day in the form of photos, video uploads, text messages, etc. A single Jet engine can generate more than 10 terabytes of data in 30 minutes of flight time. With many thousand flights per day, generation of data reaches up to many petabytes. Data contained in these sets are not always structured. It can be semi-structured or even unstructured. When such is the size and dimension of data we can well imagine how complicated it must be

In [3]:
with open('positive-words.txt', 'r') as f:
    positive_words = f.read().splitlines()

with open('negative-words.txt', 'r') as f:
    negative_words = f.read().splitlines()
    

words = article.lower().split()

pscore = 0
nscore = 0
for word in words:
    if word in positive_words:
        pscore += 1
    elif word in negative_words:
        nscore += 1
        
new_nscore= -1*nscore

#printing positive and negetive score
print(pscore)
print(new_nscore)

42
-37


In [4]:
#polarity score
polarity_score = (pscore-nscore)/((pscore+nscore)+0.000001)
print(polarity_score)

0.06329113843935268


In [5]:
#subjectivity score
from textblob import TextBlob
blob = TextBlob(article)
subjectivity = blob.sentiment.subjectivity
print(subjectivity)

0.4590204191089148


In [6]:
#average sentence length
import nltk
nltk.download('punkt')
sentences = nltk.sent_tokenize(article)
num_sentences = len(sentences)
num_words = len(nltk.word_tokenize(article))

avg_sentence_length = num_words / num_sentences
print(round(avg_sentence_length))


17


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
import re
syllable_regex = re.compile(r'\b\w{3,}\b', re.IGNORECASE)
#complex words
syllable_count = sum(len(re.findall(r'[aeiouy]+', word, re.IGNORECASE)) for word in syllable_regex.findall(article))
#words with atleast 3 syllables
complex_word_count = sum(1 for word in syllable_regex.findall(article) if len(re.findall(r'[aeiouy]+', word, re.IGNORECASE)) >= 3)

print(complex_word_count)
print(syllable_count)


249
1722


In [8]:
#complex word percentage
complexwords_per= complex_word_count/len(article.split())
print(round(complexwords_per*100))

24


In [9]:
#fog index
fog_in = 0.4*(avg_sentence_length/complexwords_per)
print((fog_in))

29.278325422903738


In [10]:
#average words per sentence
import nltk
sentences = nltk.sent_tokenize(article)
words = nltk.word_tokenize(article)

average_words_per_sentence = len(words) / len(sentences)

print(round(average_words_per_sentence))

17


In [11]:
#clean data
from nltk.corpus import stopwords
stopword_rem = set(stopwords.words('english'))
punc_rem = set(string.punctuation)
words_cleaned = [word for word in words if word.lower() not in stopword_rem and word.lower() not in punc_rem]
clean_txt = " ".join(words_cleaned)
#word count
print(len(clean_txt))

4651


In [12]:
#count number of syllable in each word
def count_syllables(word):
    vowels = 'aeiouy'
    exceptions = ['ia', 'io', 'ie', 'iou', 'ai', 'au', 'ea', 'eo', 'eu', 'oi', 'ou']
    syllables = 0
    prev_char_vowel = False
    for char in word.lower():
        if char in vowels and not prev_char_vowel:
            syllables += 1
            prev_char_vowel = True
        elif char not in vowels:
            prev_char_vowel = False
    for exception in exceptions:
        if exception in word.lower():
            syllables -= 1
    if len(word) > 2 and word[-2:] == 'es':
        syllables -= 1
    elif len(word) > 2 and word[-2:] == 'ed':
        syllables -= 1
    elif len(word) > 1 and word[-1] == 'e':
        syllables -= 1
    return syllables

In [13]:
#avrg syllable count in each word
word=re.findall(r'\b\w+\b', article)
syllables_per_word = [count_syllables(word) for word in words]

avrg_syllable_count = sum(syllables_per_word)/len(syllables_per_word)
print(round(avrg_syllable_count))

1


In [14]:
#calculate personal pronouns
pronounRegex = re.compile(r'I|we|my|ours|us',re.IGNORECASE)
pronouns = pronounRegex.findall(article)
print(len(pronouns))

467


In [15]:
#average word length
words=article.lower().split()
avrg_word_length=sum(len(word) for word in words)/len(words)
print(round(avrg_word_length))

    

5


In [16]:
print(f'''
        positive score : {pscore}
        negetive score : {nscore}
        polarity score : {polarity_score}
        subjectivity score : {subjectivity}
        average sentence length : {round(avg_sentence_length)}
        percentage of complex words : {round(complexwords_per*100)}%
        fog index : {round(fog_in)}
        average number of words in a sentence : {round(average_words_per_sentence)}
        complex word count : {complex_word_count}
        word count : {len(clean_txt)}
        syllable per word : {round(avrg_syllable_count)}
        personal pronoun : {len(pronouns)}
        average word length : {round(avrg_word_length)} 
      ''')


        positive score : 42
        negetive score : 37
        polarity score : 0.06329113843935268
        subjectivity score : 0.4590204191089148
        average sentence length : 17
        percentage of complex words : 24%
        fog index : 29
        average number of words in a sentence : 17
        complex word count : 249
        word count : 4651
        syllable per word : 1
        personal pronoun : 467
        average word length : 5 
      


In [17]:
list=[pscore, nscore, polarity_score, subjectivity, round(avg_sentence_length), round(complexwords_per*100), round(fog_in), round(average_words_per_sentence), complex_word_count, len(clean_txt), round(avrg_syllable_count), len(pronouns), round(avrg_word_length)] 
print(list)   

[42, 37, 0.06329113843935268, 0.4590204191089148, 17, 24, 29, 17, 249, 4651, 1, 467, 5]


In [18]:
#there is an exception or a problem with this module ..
#after saving one file you have to delete the file for operation of the next url 
import xlwt

wb = xlwt.Workbook()
ws = wb.add_sheet('Sheet 1')

first_row=0

for index, item in enumerate(list):
        ws.write(first_row, index, item) 

wb.save('example.xls')